In [39]:
import numpy as np
import math
from IPython.core.debugger import set_trace

lowerBound, upperBound = -5, 10
x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
N = len(x)


def f(x):
    #Sphere
    Sum = 0
    for i in range(0,N):
        Sum = Sum + (x[i])**2

    #Rosenbrock
    #Sum = 0
    #for i in range(0, N-1):
    #    Sum = Sum + 100 * ((x[i])**2 - x[i+1])**2 + (x[i] - 1)**2
        
    #Ackley
    #Sum1, Sum2 = 0, 0
    #for i in range(0,N):
    #    Sum1 = Sum1 + (x[i])**2
    #    Sum2 = Sum2 + math.cos(2*math.pi*x[i])
    #Sum= -20.0*math.exp(-0.2*math.sqrt((1/N)*(Sum1)))-math.exp((1/N) * Sum2) + math.e + 20
    return Sum

l = [lowerBound for i in range(10)]
u = [upperBound for i in range(10)]

In [40]:
def _e(index,size):
    arr = np.zeros(size)
    arr[index] = 1.0
    return arr

In [41]:
def Delta(x):
    Sum = 0
    #no general, only box constraints
    for i in range(0,N):
        Sum = Sum + max(x[i]-u[i],0) + max(l[i]-x[i], 0)
    return Sum

In [42]:
def better(x,y):
    Dx = Delta(x)
    Dy = Delta(y)
    if(Dx < Dy):
        return True
    elif(Dx == Dy and f(x) < f(y)):
        return True
    else:
        return False

In [43]:
def findExtremes(s):
    Ds = np.empty([N+1])
    fs = np.empty([N+1])
    
    for i in range(0, N+1):
        Ds[i] = Delta(s[i])
        fs[i] = f(s[i])
    
    arg_b = np.argmin(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_b] and i != arg_b:
            if(fs[i] < fs[arg_b]):
                arg_b = i;
                
    arg_w = np.argmax(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_w] and i != arg_w:
            if(fs[i] > fs[arg_w]):
                arg_w = i;
    
    #mask the worst to find the second worst
    Ds[arg_w] = 0
    
    arg_sw = np.argmax(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_sw] and i != arg_w:
            if(fs[i] > fs[arg_sw]):
                arg_sw = i;
    
    return arg_b,arg_w,arg_sw

In [44]:
def centroid(s,argWorst):
    return np.mean(np.delete(s, argWorst), axis = 0)

In [45]:
def SimplexLocalSearch(x,Epsilon,Lambda,M,Alpha=1,Gamma=2,Beta=-0.5):
    N = len(x)
    s = np.empty([N+1,N])
    s[0] = x
    for j in range(1,N+1):
        s[j] = x + Lambda*(u[j-1]-l[j-1])*_e(j-1,N);
    k = 0
    
    
    argBest, argWorst, argSecondWorst = findExtremes(s)
    sBest, sWorst, sSecondWorst = s[argBest], s[argWorst], s[argSecondWorst]
    #set_trace()
    while abs(f(sBest) - f(sWorst)) > Epsilon and k < M:
        #set_trace()
        #if(k%1000 == 0):print(f(sBest),'\n')
        r = (1+Alpha)*centroid(s,argWorst) - Alpha*sWorst
        if(better(sBest,r) and better(r,sSecondWorst)):
            s[argWorst] = r
        elif(better(r,sBest)):
            e = (1+Gamma)*centroid(s,argWorst)-Gamma*sWorst
            if(better(e,r)):
                s[argWorst] = e
            else:
                s[argWorst] = r
        else:
            c = (1+Beta)*centroid(s,argWorst)-Beta*sWorst
            if(better(c,sWorst)):
                s[argWorst] = c
            else:
                for j in range(0,N+1):
                    s[j] = (s[j] + sBest)/2

        #set_trace()
        argBest, argWorst, argSecondWorst = findExtremes(s)
        sBest, sWorst, sSecondWorst = s[argBest], s[argWorst], s[argSecondWorst]
        k = k + 1
    #print(f(sBest))
    return sBest, k, s, argBest, argWorst, argSecondWorst

In [46]:
def DirectionalEscape(Epsilon,EpsilonApostrophe,Lambda,M, Gamma = 2,Beta=-0.5):
    k = 0
    it = 0
    x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
    xWorse = x
    x, _k, s, argBest, argWorst, argSecondWorst = SimplexLocalSearch(x,Epsilon,Lambda,M-k)
    xBest = x
    while(k < M):
        #set_trace()
        k = k + _k
        if k >= M:
            break
        
        if f(x) < f(xBest):
            print("***NEW BEST VALUE*** iteration: ", k, "| best found value: ", f(x),"| old value found: ", f(xBest))
            xBest = x
            
        if( k//1000 > it):
            it = k//1000
            print("iteration:", it * 1000, "| best found value: ", f(xBest), "but x is here: ", f(x))
        first = True
        
        while(first or better(x, xWorse) ):
            first = False
            xWorse = x
            c = (1+Beta)*centroid(s,argWorst)-Beta*s[argWorst]
            x = Gamma*s[argSecondWorst] + (1-Gamma) * c
            s[argBest] = x
        x, _k, s, argBest, argWorst, argSecondWorst = SimplexLocalSearch(x,Epsilon,Lambda,M-k)
        
    return xBest

In [47]:
Lambda = 1
M = 10000
Epsilon = 10e-1
#set_trace()
EpsilonApostrophe = 0
print(DirectionalEscape(Epsilon,EpsilonApostrophe,Lambda,M))

***NEW BEST VALUE*** iteration:  276 | best found value:  3.596450035715992 | old value found:  29.75486558939278
***NEW BEST VALUE*** iteration:  357 | best found value:  0.7781786818477455 | old value found:  6.428547851019348
***NEW BEST VALUE*** iteration:  425 | best found value:  0.5767353484554073 | old value found:  6.423676956646863
***NEW BEST VALUE*** iteration:  489 | best found value:  0.633560628479123 | old value found:  3.2600290848545703
***NEW BEST VALUE*** iteration:  556 | best found value:  0.7728859413120039 | old value found:  3.79967365626889
***NEW BEST VALUE*** iteration:  618 | best found value:  0.7735699716839863 | old value found:  4.140265197514593
***NEW BEST VALUE*** iteration:  683 | best found value:  0.5415996123906219 | old value found:  2.6631271283128184
***NEW BEST VALUE*** iteration:  748 | best found value:  0.7769940248787391 | old value found:  3.9477058538631615
***NEW BEST VALUE*** iteration:  813 | best found value:  0.5816627301753904 | o

***NEW BEST VALUE*** iteration:  4847 | best found value:  0.7752690824967351 | old value found:  3.2151820959283337
***NEW BEST VALUE*** iteration:  4909 | best found value:  0.6372082433068387 | old value found:  3.8073821283035363
***NEW BEST VALUE*** iteration:  4972 | best found value:  0.6148595431429107 | old value found:  4.671266419909293
***NEW BEST VALUE*** iteration:  5036 | best found value:  0.6230395130499244 | old value found:  2.528233535499418
iteration: 5000 | best found value:  0.6230395130499244 but x is here:  0.6230395130499244
***NEW BEST VALUE*** iteration:  5106 | best found value:  0.7531191643318442 | old value found:  4.16227630412361
***NEW BEST VALUE*** iteration:  5170 | best found value:  0.5766362705730118 | old value found:  2.6530857438174476
***NEW BEST VALUE*** iteration:  5225 | best found value:  3.2804346667495166 | old value found:  4.007820845801382
***NEW BEST VALUE*** iteration:  5312 | best found value:  0.47693017639219015 | old value foun

***NEW BEST VALUE*** iteration:  9395 | best found value:  3.0007819382095615 | old value found:  3.7915164183100973
***NEW BEST VALUE*** iteration:  9467 | best found value:  0.4658221441252321 | old value found:  8.502152851210468
***NEW BEST VALUE*** iteration:  9532 | best found value:  0.6897627723395512 | old value found:  3.678815315316546
***NEW BEST VALUE*** iteration:  9597 | best found value:  0.639045935223999 | old value found:  3.216790352704064
***NEW BEST VALUE*** iteration:  9652 | best found value:  3.1216576900603563 | old value found:  3.5613719739665117
***NEW BEST VALUE*** iteration:  9727 | best found value:  0.5176072063298354 | old value found:  15.674106376766073
***NEW BEST VALUE*** iteration:  9928 | best found value:  0.6786146748525179 | old value found:  3.04235333134067
***NEW BEST VALUE*** iteration:  9999 | best found value:  0.6849387024788759 | old value found:  3.6833244309027795


NameError: name 'xBest' is not defined